In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        os.path.join(dirname, filename)

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Conv2D, MaxPool2D, BatchNormalization, GlobalAveragePooling2D, Flatten, ZeroPadding2D, Rescaling
from keras.preprocessing import image
from keras.applications.imagenet_utils import preprocess_input

In [5]:
train_df = pd.read_csv("/kaggle/input/humpback-whale-identification/train.csv")

In [6]:
train_df.shape

(25361, 2)

In [7]:
d = dict(enumerate(train_df['Id'].astype('category').cat.categories))
#d

In [8]:
train_df['Id'] = train_df['Id'].astype('category').cat.codes

In [9]:
test_df = pd.read_csv("/kaggle/input/humpback-whale-identification/sample_submission.csv")

In [10]:
train_df.head()

,Image,Id
0,0000e88ab.jpg,4786
1,0001f9222.jpg,3808
2,00029d126.jpg,662
3,00050a15a.jpg,0
4,0005c1ef8.jpg,0


In [11]:
INP_SIZE      = (512, 512) 
TARGET_SIZE   = (150, 150) 
INTERPOLATION = "bilinear"
N_CLASSES = 5005

NUM_FOLDS  = 5
BATCH_SIZE = 128
SEED       = 42

DATA_DIR  = '../input/humpback-whale-identification/'
TRAIN_DIR = DATA_DIR + 'train/'
TEST_DIR  = DATA_DIR + 'test/'

In [12]:
# SetAutoTune
AUTOTUNE = tf.data.AUTOTUNE 
#https://www.kaggle.com/ipythonx/tf-keras-learning-to-resize-image-for-vit-model/notebook
def build_augmenter(is_labelled):
    def augment(img):
        img = tf.image.random_flip_left_right(img)
        img = tf.image.random_flip_up_down(img)
        img = tf.image.random_saturation(img, 0.65, 1.05)
        img = tf.image.random_brightness(img, 0.05)
        img = tf.image.random_contrast(img, 0.75, 1.05)
        img = tf.image.random_hue(img, 0.05)
        return img
    
    def augment_with_labels(img, label):
        return augment(img), label
    return augment_with_labels if is_labelled else augment

def build_decoder(is_labelled, size):
    def decode(path):
        file_bytes = tf.io.read_file(path)
        img = tf.image.decode_jpeg(file_bytes, channels = 1)
        img = tf.image.resize(img, (size[0], size[1]))
        return img
    
    def decode_with_labels(path, label):
        label = tf.cast(label, tf.int32)
        return decode(path),label
    
    return decode_with_labels if is_labelled else decode

def create_dataset(df, 
                   batch_size  = 64, 
                   is_labelled = False, 
                   augment     = False, 
                   repeat      = False, 
                   shuffle     = False,
                   size        = TARGET_SIZE):
    decode_fn    = build_decoder(is_labelled, size)
    augmenter_fn = build_augmenter(is_labelled)
    
    # Create Dataset
    if is_labelled:
        dataset = tf.data.Dataset.from_tensor_slices((df['Image'].values, df['Id'].values))
    else:
        dataset = tf.data.Dataset.from_tensor_slices((df['Image'].values))
        
    dataset = dataset.map(decode_fn, num_parallel_calls = AUTOTUNE)
    dataset = dataset.map(augmenter_fn, num_parallel_calls = AUTOTUNE) if augment else dataset
    dataset = dataset.repeat(batch_size) if repeat else dataset
    dataset = dataset.shuffle(1024, reshuffle_each_iteration = True) if shuffle else dataset
    dataset = dataset.batch(batch_size)
    dataset = dataset.cache().prefetch(buffer_size=AUTOTUNE)
    return dataset

In [13]:
train_df['Image'] = train_df['Image'].apply(lambda x: f'{TRAIN_DIR}{x}')

In [14]:
train_ds = create_dataset(train_df, batch_size = BATCH_SIZE, is_labelled = True, repeat = True)

2022-08-15 08:56:20.763958: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [15]:
for image_batch, labels_batch in train_ds:
  print(image_batch.shape)
  print(labels_batch.shape)
  break

2022-08-15 08:56:20.982024: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


(128, 150, 150, 1)
(128,)


2022-08-15 08:56:21.543435: W tensorflow/core/kernels/data/cache_dataset_ops.cc:768] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


In [16]:
test_df['Image'] = test_df['Image'].apply(lambda x: f'{TEST_DIR}{x}')
test_ds = create_dataset(test_df, batch_size = BATCH_SIZE)

In [17]:
for image_batch in test_ds:
  print(image_batch.shape)
  break

(128, 150, 150, 1)


2022-08-15 08:56:21.993059: W tensorflow/core/kernels/data/cache_dataset_ops.cc:768] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


In [35]:
model = tf.keras.Sequential([
    Rescaling(1./255, input_shape=(150, 150,1)),
    Conv2D(32, (3,3),1,padding='same', activation='relu'),
    BatchNormalization(axis=1, momentum = 0.99, epsilon=0.001),
    MaxPool2D((2,2)),
    
    Conv2D(64, (3,3),1,padding='same', activation='relu'),
    BatchNormalization(axis=1, momentum = 0.99, epsilon=0.001),
    MaxPool2D((2,2)),
    
    GlobalAveragePooling2D(),
    BatchNormalization(axis=1, momentum = 0.99, epsilon=0.001),
    Dense(N_CLASSES, activation='sigmoid')
])

In [36]:
model.compile(
 loss='sparse_categorical_crossentropy', metrics=[tf.keras.metrics.TopKCategoricalAccuracy(k=5)], optimizer="adam")

In [37]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
rescaling_3 (Rescaling)      (None, 150, 150, 1)       0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 150, 150, 32)      320       
_________________________________________________________________
batch_normalization_9 (Batch (None, 150, 150, 32)      600       
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 75, 75, 32)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 75, 75, 64)        18496     
_________________________________________________________________
batch_normalization_10 (Batc (None, 75, 75, 64)        300       
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 37, 37, 64)       

In [38]:
checkpoint_path = "/kaggle/working/mymodel/training_2/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)
callbacks = [
    tf.keras.callbacks.ModelCheckpoint(
        # Path where to save the model
        # The two parameters below mean that we will overwrite
        # the current checkpoint if and only if
        # the `val_loss` score has improved.
        # The saved model name will include the current epoch.
        filepath= checkpoint_path,
        verbose=1,
        save_weights_only=True
    )
]

In [39]:
model.save_weights(checkpoint_path.format(epoch=0))

In [41]:
history = model.fit(
  train_ds,
  epochs=10,
  steps_per_epoch = 64,
  callbacks=callbacks
)

Epoch 1/10
64/64 [==============================] - 118s 2s/step - loss: 8.0103 - top_k_categorical_accuracy: 0.5317

Epoch 00001: saving model to /kaggle/working/mymodel/training_2/cp-0001.ckpt
Epoch 2/10
59/64 [==========================>...] - ETA: 9s - loss: 7.4824 - top_k_categorical_accuracy: 0.6373 

KeyboardInterrupt: 

In [ ]:
os.listdir(checkpoint_dir)

In [ ]:
latest = tf.train.latest_checkpoint(checkpoint_dir)

In [ ]:
model.load_weights(latest)

In [ ]:
preds = model.predict(test_ds)

In [ ]:
len(preds)

In [ ]:
preds.shape

In [ ]:
submission_df = pd.read_csv("/kaggle/input/humpback-whale-identification/sample_submission.csv")


In [ ]:
for i, pred in enumerate(preds):
    arr = pred.argsort()[-5:][::-1]
    submission_df.loc[i, 'Id'] = " ".join(np.vectorize(d.get)(arr))

In [ ]:
submission_df.to_csv('submission.csv', index = False)
print(submission_df.shape)